# Integrator Testbed

Experiment with the core integrator across all supported simulation types. Toggle the legacy
comparison when needed, tune particle parameters, and explore output artefacts for debugging or validation.

In [1]:
from __future__ import annotations

import html
import io
import json
import sys
from pathlib import Path
from typing import Dict, List, Optional

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

# Ensure repository paths are importable when running in VS Code/notebooks
PROJECT_ROOT = Path.cwd().resolve()
while PROJECT_ROOT.name and PROJECT_ROOT.name != "LW_windows":
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

LEGACY_ROOT = PROJECT_ROOT / "legacy"
if str(LEGACY_ROOT) not in sys.path:
    sys.path.insert(0, str(LEGACY_ROOT))

VALIDATION_ROOT = PROJECT_ROOT / "examples" / "validation"
if str(VALIDATION_ROOT) not in sys.path:
    sys.path.insert(0, str(VALIDATION_ROOT))

from core_vs_legacy_benchmark import (
    DEFAULT_DRIVER_PARAMS,
    DEFAULT_RIDER_PARAMS,
    PARTICLE_PARAM_FIELDS,
    SimulationType,
    compute_delta_energy_series,
    prepare_two_particle_demo,
    run_benchmark,
    summarise_metrics,
 )

COLOR_RIDER = "#0072B2"
COLOR_DRIVER = "#D55E00"
COLOR_LEGACY_RIDER = "#56B4E9"
COLOR_LEGACY_DRIVER = "#E69F00"
COLOR_DIFF_RIDER = "#009E73"
COLOR_DIFF_DRIVER = "#CC79A7"
SCATTER_STYLE = {"s": 140, "alpha": 0.78, "linewidth": 0, "edgecolors": "none"}
TITLE_FONTSIZE = 18
LABEL_FONTSIZE = 16
TICK_FONTSIZE = 13
LEGEND_FONTSIZE = 12
AVAILABLE_DPI_CHOICES = (150, 300, 450, 600)
DEFAULT_PLOT_DPI = 300

LAST_METRICS: Optional[Dict[str, Dict[str, float]]] | None = None
LAST_PAYLOAD: Optional[dict] = None
LAST_LOG_MESSAGES: List[str] = []

plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams.update(
    {
        "axes.titlesize": TITLE_FONTSIZE,
        "axes.labelsize": LABEL_FONTSIZE,
        "xtick.labelsize": TICK_FONTSIZE,
        "ytick.labelsize": TICK_FONTSIZE,
        "legend.fontsize": LEGEND_FONTSIZE,
    }
)

In [2]:
PARAM_LABELS = {
    "starting_distance": "Start z (mm)",
    "transv_mom": "Transverse momentum (amu·mm/ns)",
    "starting_Pz": "Initial Pz (amu·mm/ns)",
    "stripped_ions": "Stripped ions",
    "m_particle": "Mass (amu)",
    "transv_dist": "Transverse spread (mm)",
    "pcount": "Particle count",
    "charge_sign": "Charge sign",
}

CORE_PARAM_LABELS = {
    "time_step": "Time step (ns)",
    "wall_z": "Wall z (mm)",
    "aperture_radius": "Aperture radius (mm)",
    "mean": "Mean separation (mm)",
    "cav_spacing": "Cavity spacing (mm)",
    "z_cutoff": "z cutoff (mm)",
}

PARTICLE_WIDGET_WIDTH = widgets.Layout(width="320px")
CORE_WIDGET_WIDTH = widgets.Layout(width="280px")
PARTICLE_WIDGET_STYLE = {"description_width": "175px"}
CORE_WIDGET_STYLE = {"description_width": "175px"}
ROW_LAYOUT = widgets.Layout(gap="14px")

SIMULATION_TYPE_OPTIONS = {
    "Conducting wall": SimulationType.CONDUCTING_WALL,
    "Switching wall": SimulationType.SWITCHING_WALL,
    "Bunch to bunch": SimulationType.BUNCH_TO_BUNCH,
}

SPECIES_PRESETS: Dict[str, Optional[Dict[str, float]]] = {
    "custom": None,
    "electron": {
        "m_particle": 5.48579909070e-4,
        "charge_sign": -1.0,
        "stripped_ions": 1.0,
    },
    "positron": {
        "m_particle": 5.48579909070e-4,
        "charge_sign": 1.0,
        "stripped_ions": 1.0,
    },
    "proton": {
        "m_particle": 1.007276466621,
        "charge_sign": 1.0,
        "stripped_ions": 1.0,
    },
    "antiproton": {
        "m_particle": 1.007276466621,
        "charge_sign": -1.0,
        "stripped_ions": 1.0,
    },
    "lead": {
        "m_particle": 207.9766521,
        "charge_sign": 1.0,
        "stripped_ions": 82.0,
    },
    "gold": {
        "m_particle": 196.9665687,
        "charge_sign": 1.0,
        "stripped_ions": 79.0,
    },
}

SPECIES_DROPDOWN_OPTIONS = [
    ("Custom / manual", "custom"),
    ("Electron (e⁻)", "electron"),
    ("Positron (e⁺)", "positron"),
    ("Proton (p⁺)", "proton"),
    ("Antiproton (p̄⁻)", "antiproton"),
    ("Lead ion (Pb⁸²⁺)", "lead"),
    ("Gold ion (Au⁷⁹⁺)", "gold"),
]

CORE_PARAM_DEFAULTS = {
    "time_step": 2.2e-7,
    "wall_z": 1e5,
    "aperture_radius": 1e5,
    "mean": 1e5,
    "cav_spacing": 1e5,
    "z_cutoff": 0.0,
}

CORE_REQUIRED_PARAMS = {
    SimulationType.CONDUCTING_WALL: {"time_step", "wall_z", "aperture_radius"},
    SimulationType.SWITCHING_WALL: {
        "time_step",
        "wall_z",
        "aperture_radius",
        "cav_spacing",
        "z_cutoff",
    },
    SimulationType.BUNCH_TO_BUNCH: {"time_step", "aperture_radius"},
}


def _make_particle_widgets(defaults: Dict[str, float | int]) -> Dict[str, widgets.Widget]:
    controls: Dict[str, widgets.Widget] = {}
    for name in PARTICLE_PARAM_FIELDS:
        default_value = defaults[name]
        description = PARAM_LABELS.get(name, name.replace("_", " ").title())
        if isinstance(default_value, int):
            control = widgets.IntText(
                value=int(default_value),
                description=description,
                layout=PARTICLE_WIDGET_WIDTH,
                style=PARTICLE_WIDGET_STYLE,
            )
        else:
            control = widgets.FloatText(
                value=float(default_value),
                description=description,
                layout=PARTICLE_WIDGET_WIDTH,
                style=PARTICLE_WIDGET_STYLE,
            )
        controls[name] = control
    return controls


def _make_core_widgets() -> Dict[str, widgets.Widget]:
    controls: Dict[str, widgets.Widget] = {}
    for name, default_value in CORE_PARAM_DEFAULTS.items():
        description = CORE_PARAM_LABELS.get(name, name.replace("_", " ").title())
        controls[name] = widgets.FloatText(
            value=float(default_value),
            description=description,
            layout=CORE_WIDGET_WIDTH,
            style=CORE_WIDGET_STYLE,
        )
    return controls


def _apply_species_preset(
    controls: Dict[str, widgets.Widget], preset_key: str
) -> None:
    preset = SPECIES_PRESETS.get(preset_key)
    if not preset:
        return
    for field, value in preset.items():
        control = controls.get(field)
        if control is None:
            continue
        current_value = control.value
        if isinstance(current_value, int):
            control.value = int(round(value))
        else:
            control.value = float(value)


def _particle_rows(controls: Dict[str, widgets.Widget]) -> List[widgets.Widget]:
    rows: List[widgets.Widget] = []
    row: List[widgets.Widget] = []
    for name in PARTICLE_PARAM_FIELDS:
        row.append(controls[name])
        if len(row) == 2:
            rows.append(widgets.HBox(row, layout=ROW_LAYOUT))
            row = []
    if row:
        rows.append(widgets.HBox(row, layout=ROW_LAYOUT))
    return rows


def _core_rows(controls: Dict[str, widgets.Widget]) -> List[widgets.Widget]:
    rows: List[widgets.Widget] = []
    row: List[widgets.Widget] = []
    for name in CORE_PARAM_DEFAULTS:
        row.append(controls[name])
        if len(row) == 3:
            rows.append(widgets.HBox(row, layout=ROW_LAYOUT))
            row = []
    if row:
        rows.append(widgets.HBox(row, layout=ROW_LAYOUT))
    return rows


def _collect_particle_values(controls: Dict[str, widgets.Widget]) -> Dict[str, float | int]:
    values: Dict[str, float | int] = {}
    for name, control in controls.items():
        values[name] = control.value
    return values


def _collect_core_values(controls: Dict[str, widgets.Widget]) -> Dict[str, float]:
    values: Dict[str, float] = {}
    for name, control in controls.items():
        values[name] = float(control.value)
    return values


def _build_particle_section(
    title: str, controls: Dict[str, widgets.Widget], preset_widget: widgets.Widget | None
) -> widgets.Accordion:
    rows = []
    if preset_widget is not None:
        rows.append(preset_widget)
    rows.extend(_particle_rows(controls))
    accordion = widgets.Accordion(children=[widgets.VBox(rows)])
    accordion.set_title(0, title)
    return accordion


def _build_core_section(controls: Dict[str, widgets.Widget]) -> widgets.Accordion:
    rows = _core_rows(controls)
    accordion = widgets.Accordion(children=[widgets.VBox(rows)])
    accordion.set_title(0, "Core configuration")
    return accordion


def _strip_driver_summary(summary: str) -> str:
    lines: List[str] = []
    skipping = False
    for line in summary.splitlines():
        if line.startswith("- Driver"):
            skipping = True
            continue
        if skipping:
            if line.startswith("  driver"):
                continue
            skipping = False
        lines.append(line)
    return "\n".join(lines)


def _required_params_for(sim_type: SimulationType) -> set[str]:
    return CORE_REQUIRED_PARAMS.get(sim_type, set())


def _apply_core_param_state(
    controls: Dict[str, widgets.Widget], required_params: set[str]
) -> None:
    for name, control in controls.items():
        control.disabled = name not in required_params
        control.layout.opacity = 1.0 if name in required_params else 0.45


In [ ]:
steps_widget = widgets.IntSlider(value=1000, min=10, max=20000, step=10, description="Steps:", continuous_update=False)
seed_widget = widgets.IntText(value=12345, description="Seed:")
legacy_toggle = widgets.Checkbox(value=False, description="Include legacy comparison")
simulation_widget = widgets.Dropdown(options=[(label, value) for label, value in SIMULATION_TYPE_OPTIONS.items()], value=SimulationType.BUNCH_TO_BUNCH, description="Core mode:")
overlay_display_widget = widgets.Checkbox(value=False, description="Show legacy overlay plot")
overlay_save_widget = widgets.Checkbox(value=False, description="Save legacy overlay plot")
difference_display_widget = widgets.Checkbox(value=False, description="Show Δ(core−legacy) plot")
difference_save_widget = widgets.Checkbox(value=False, description="Save Δ(core−legacy) plot")
metrics_save_widget = widgets.Checkbox(value=False, description="Save metrics JSON")
energy_save_widget = widgets.Checkbox(value=False, description="Save ΔE plots")
dpi_widget = widgets.Dropdown(
    options=[(f"{value} dpi", value) for value in AVAILABLE_DPI_CHOICES],
    value=DEFAULT_PLOT_DPI,
    description="Plot DPI:",
    layout=widgets.Layout(width="260px"),
    style=dict(CORE_WIDGET_STYLE, description_width="120px"),
)
output_dir_widget = widgets.Text(value="test_outputs/testbed_runs", description="Plot dir:", layout=widgets.Layout(width="360px"), style=CORE_WIDGET_STYLE)
config_dir_widget = widgets.Text(value="configs/testbed_runs", description="Config dir:", layout=widgets.Layout(width="360px"), style=CORE_WIDGET_STYLE)
config_name_widget = widgets.Text(value="testbed_config.json", description="Config name:", layout=widgets.Layout(width="320px"), style=CORE_WIDGET_STYLE)
config_file_dropdown = widgets.Dropdown(
    options=[("No saved configs", "")],
    value="",
    description="Existing:",
    layout=widgets.Layout(width="320px"),
    style=dict(CORE_WIDGET_STYLE, description_width="120px"),
    disabled=True,
)
config_refresh_button = widgets.Button(description="Refresh list", icon="refresh")
save_config_button = widgets.Button(description="Save config", icon="save")
load_config_button = widgets.Button(description="Load config", icon="folder-open")
run_button = widgets.Button(description="Run integrator", icon="play", button_style="primary")
output_area = widgets.Output()
config_feedback_output = widgets.Output(layout=widgets.Layout(border="1px solid #dddddd", padding="8px", margin="4px 0"))
initial_state_output = widgets.Output(layout=widgets.Layout(border="1px solid #dddddd", padding="8px", margin="6px 0"))
rider_controls = _make_particle_widgets(DEFAULT_RIDER_PARAMS)
driver_controls = _make_particle_widgets(DEFAULT_DRIVER_PARAMS)
core_controls = _make_core_widgets()
rider_species_widget = widgets.Dropdown(options=SPECIES_DROPDOWN_OPTIONS, value="custom", description="Rider species:", layout=PARTICLE_WIDGET_WIDTH, style=PARTICLE_WIDGET_STYLE)
driver_species_widget = widgets.Dropdown(options=SPECIES_DROPDOWN_OPTIONS, value="custom", description="Driver species:", layout=PARTICLE_WIDGET_WIDTH, style=PARTICLE_WIDGET_STYLE)
rider_section = _build_particle_section("Rider particle", rider_controls, rider_species_widget)
driver_section = _build_particle_section("Driver particle", driver_controls, driver_species_widget)
core_section = _build_core_section(core_controls)

def _resolved_output_dir() -> Path:
    raw_value = output_dir_widget.value.strip() or "test_outputs/testbed_runs"
    return Path(raw_value).expanduser()

def _resolved_config_dir() -> Path:
    raw_value = config_dir_widget.value.strip() or "configs/testbed_runs"
    return Path(raw_value).expanduser()

def _save_dir_if_needed(should_create: bool) -> Path:
    directory = _resolved_output_dir()
    if should_create:
        directory.mkdir(parents=True, exist_ok=True)
    return directory

def _ensure_config_dir() -> Path:
    directory = _resolved_config_dir()
    directory.mkdir(parents=True, exist_ok=True)
    return directory

def _list_config_files(directory: Path) -> List[str]:
    if not directory.exists():
        return []
    return sorted(str(path.name) for path in directory.glob("*.json") if path.is_file())

def _maybe_append_json(name: str) -> str:
    candidate = name.strip() or "testbed_config.json"
    return candidate if candidate.lower().endswith(".json") else f"{candidate}.json"

def _refresh_config_file_options(selected: Optional[str] = None) -> None:
    directory = _resolved_config_dir()
    files = _list_config_files(directory)
    if files:
        options = [(file_name, file_name) for file_name in files]
        config_file_dropdown.options = options
        config_file_dropdown.disabled = False
        if selected and selected in files:
            config_file_dropdown.value = selected
        elif config_file_dropdown.value in files:
            config_file_dropdown.value = config_file_dropdown.value
        else:
            config_file_dropdown.value = options[0][1]
    else:
        config_file_dropdown.options = [("No saved configs", "")]
        config_file_dropdown.value = ""
        config_file_dropdown.disabled = True

def _update_legacy_controls(change) -> None:
    legacy_enabled = bool(change["new"] if isinstance(change, dict) else legacy_toggle.value)
    for checkbox in (overlay_display_widget, overlay_save_widget, difference_display_widget, difference_save_widget, metrics_save_widget):
        checkbox.disabled = not legacy_enabled
        if not legacy_enabled:
            checkbox.value = False

def _supports_driver(sim_type: SimulationType) -> bool:
    return sim_type == SimulationType.BUNCH_TO_BUNCH

def _apply_driver_visibility(sim_value: SimulationType) -> None:
    supports_driver = _supports_driver(sim_value)
    driver_section.layout.display = "" if supports_driver else "none"
    if not supports_driver:
        driver_section.selected_index = None

def _refresh_initial_properties(change=None) -> None:
    sim_type_value = simulation_widget.value
    supports_driver = _supports_driver(sim_type_value)
    rider_params = _collect_particle_values(rider_controls)
    driver_params = _collect_particle_values(driver_controls) if supports_driver else None
    try:
        rider_state, driver_state, rider_rest_mev, driver_rest_mev = prepare_two_particle_demo(
            seed=seed_widget.value,
            rider_params=rider_params,
            driver_params=driver_params,
        )
    except Exception as exc:
        with initial_state_output:
            initial_state_output.clear_output()
            print(f"Failed to compute initial states: {exc}")
        return

    rider_gamma = float(rider_state["gamma"][0])
    rider_rest_gev = rider_rest_mev * 1e-3
    rider_total_gev = rider_gamma * rider_rest_gev

    rows = [
        ("Rider γ", f"{rider_gamma:.6f}"),
        ("Rider rest energy", f"{rider_rest_gev:.6f} GeV ({rider_rest_mev:.2f} MeV)"),
        ("Rider total energy", f"{rider_total_gev:.6f} GeV"),
    ]

    if supports_driver and driver_params is not None:
        driver_gamma = float(driver_state["gamma"][0])
        driver_rest_gev = driver_rest_mev * 1e-3
        driver_total_gev = driver_gamma * driver_rest_gev
        rows.extend(
            [
                ("Driver γ", f"{driver_gamma:.6f}"),
                ("Driver rest energy", f"{driver_rest_gev:.6f} GeV ({driver_rest_mev:.2f} MeV)"),
                ("Driver total energy", f"{driver_total_gev:.6f} GeV"),
            ]
        )
        driver_note = ""
    else:
        driver_note = "<p style='margin:4px 0 0'><em>Driver not active for this simulation type.</em></p>"

    table_rows = "".join(
        f"<tr><th style='text-align:left;padding-right:12px'>{label}</th><td>{value}</td></tr>" for label, value in rows
    )
    table_html = (
        "<p style='margin:0 0 6px'><strong>Seed:</strong> "
        f"{seed_widget.value}</p><table style='border-collapse:collapse'>"
        f"{table_rows}</table>{driver_note}"
    )
    with initial_state_output:
        initial_state_output.clear_output()
        display(widgets.HTML(value=table_html))

def _collect_configuration_snapshot() -> dict:
    sim_type_value = simulation_widget.value
    supports_driver = _supports_driver(sim_type_value)
    rider_params = _collect_particle_values(rider_controls)
    driver_params = _collect_particle_values(driver_controls) if supports_driver else None
    core_params = _collect_core_values(core_controls)
    snapshot = {
        "steps": steps_widget.value,
        "seed": seed_widget.value,
        "simulation_type": sim_type_value.name,
        "legacy_enabled": legacy_toggle.value,
        "overlay_display": overlay_display_widget.value,
        "overlay_save": overlay_save_widget.value,
        "difference_display": difference_display_widget.value,
        "difference_save": difference_save_widget.value,
        "metrics_save": metrics_save_widget.value,
        "energy_save": energy_save_widget.value,
        "plot_dpi": dpi_widget.value,
        "output_dir": output_dir_widget.value,
        "config_dir": config_dir_widget.value,
        "config_name": config_name_widget.value,
        "rider_params": rider_params,
        "driver_params": driver_params,
        "core_params": core_params,
    }
    return snapshot

def _apply_configuration_snapshot(snapshot: dict) -> None:
    with config_feedback_output:
        config_feedback_output.clear_output()
    try:
        if "simulation_type" in snapshot:
            sim_name = snapshot["simulation_type"]
            if isinstance(sim_name, str) and hasattr(SimulationType, sim_name):
                simulation_widget.value = getattr(SimulationType, sim_name)
        if "steps" in snapshot:
            steps_widget.value = int(snapshot["steps"])
        if "seed" in snapshot:
            seed_widget.value = int(snapshot["seed"])
        if "legacy_enabled" in snapshot:
            legacy_toggle.value = bool(snapshot["legacy_enabled"])
        for widget, key in (
            (overlay_display_widget, "overlay_display"),
            (overlay_save_widget, "overlay_save"),
            (difference_display_widget, "difference_display"),
            (difference_save_widget, "difference_save"),
            (metrics_save_widget, "metrics_save"),
            (energy_save_widget, "energy_save"),
):
            if key in snapshot:
                widget.value = bool(snapshot[key])
        if "plot_dpi" in snapshot:
            dpi_widget.value = int(snapshot["plot_dpi"])
        if "output_dir" in snapshot:
            output_dir_widget.value = str(snapshot["output_dir"])
        if "config_dir" in snapshot:
            config_dir_widget.value = str(snapshot["config_dir"])
        if "config_name" in snapshot:
            config_name_widget.value = str(snapshot["config_name"])
        rider_params = snapshot.get("rider_params") or {}
        for name, control in rider_controls.items():
            if name in rider_params:
                control.value = rider_params[name]
        driver_params = snapshot.get("driver_params") or {}
        for name, control in driver_controls.items():
            if name in driver_params:
                control.value = driver_params[name]
        core_params = snapshot.get("core_params") or {}
        for name, control in core_controls.items():
            if name in core_params:
                control.value = core_params[name]
        _refresh_initial_properties()
        with config_feedback_output:
            print("Configuration loaded.")
    except Exception as exc:
        with config_feedback_output:
            print(f"Failed to load configuration: {exc}")

def _handle_save_config(_):
    snapshot = _collect_configuration_snapshot()
    try:
        target_dir = _ensure_config_dir()
        file_name = _maybe_append_json(snapshot.get("config_name", "testbed_config.json"))
        target_path = target_dir / file_name
        payload = dict(snapshot)
        payload.pop("config_dir", None)
        payload.pop("config_name", None)
        with target_path.open("w", encoding="utf-8") as fh:
            json.dump(payload, fh, indent=2, sort_keys=True)
        with config_feedback_output:
            config_feedback_output.clear_output()
            print(f"Configuration saved to {target_path}")
        config_name_widget.value = file_name
        _refresh_config_file_options(selected=file_name)
    except Exception as exc:
        with config_feedback_output:
            config_feedback_output.clear_output()
            print(f"Failed to save configuration: {exc}")

def _handle_load_config(_):
    try:
        target_dir = _resolved_config_dir()
        file_name = _maybe_append_json(config_name_widget.value)
        target_path = target_dir / file_name
        if not target_path.exists():
            raise FileNotFoundError(f"No config found at {target_path}")
        with target_path.open("r", encoding="utf-8") as fh:
            payload = json.load(fh)
        rebuilt_snapshot = dict(payload)
        rebuilt_snapshot["config_dir"] = str(config_dir_widget.value)
        rebuilt_snapshot["config_name"] = file_name
        _apply_configuration_snapshot(rebuilt_snapshot)
        _refresh_config_file_options(selected=file_name)
    except Exception as exc:
        with config_feedback_output:
            config_feedback_output.clear_output()
            print(f"Failed to load configuration: {exc}")

def _on_rider_species(change) -> None:
    new_value = change.get("new") if isinstance(change, dict) else rider_species_widget.value
    if not new_value or new_value == change.get("old"):
        return
    _apply_species_preset(rider_controls, str(new_value))
    _refresh_initial_properties()

def _on_driver_species(change) -> None:
    new_value = change.get("new") if isinstance(change, dict) else driver_species_widget.value
    if not new_value or new_value == change.get("old"):
        return
    _apply_species_preset(driver_controls, str(new_value))
    _refresh_initial_properties()

def _on_config_dir_change(change) -> None:
    _refresh_config_file_options()

def _on_config_file_selected(change) -> None:
    new_value = change.get("new")
    if not new_value:
        return
    config_name_widget.value = new_value

rider_species_widget.observe(_on_rider_species, names="value")
driver_species_widget.observe(_on_driver_species, names="value")
legacy_toggle.observe(lambda change: _update_legacy_controls(change), names="value")
_update_legacy_controls({"new": legacy_toggle.value})

def _update_core_controls(change=None) -> None:
    sim_value = change["new"] if isinstance(change, dict) else simulation_widget.value
    required = _required_params_for(sim_value)
    _apply_core_param_state(core_controls, required)
    _apply_driver_visibility(sim_value)
    _refresh_initial_properties()

simulation_widget.observe(_update_core_controls, names="value")
_update_core_controls()

seed_widget.observe(_refresh_initial_properties, names="value")
for control in rider_controls.values():
    control.observe(_refresh_initial_properties, names="value")
for control in driver_controls.values():
    control.observe(_refresh_initial_properties, names="value")
_refresh_initial_properties()
config_dir_widget.observe(_on_config_dir_change, names="value")
config_file_dropdown.observe(_on_config_file_selected, names="value")
config_refresh_button.on_click(lambda _: _refresh_config_file_options())
_refresh_config_file_options()

save_config_button.on_click(_handle_save_config)
load_config_button.on_click(_handle_load_config)

def handle_run(_):
    with output_area:
        output_area.clear_output(wait=True)
        legacy_enabled = legacy_toggle.value
        sim_type_value = simulation_widget.value
        supports_driver = _supports_driver(sim_type_value)
        rider_params = _collect_particle_values(rider_controls)
        driver_params = _collect_particle_values(driver_controls) if supports_driver else None
        core_params = _collect_core_values(core_controls)
        required_core = _required_params_for(sim_type_value)
        needs_dir = any([
            overlay_save_widget.value,
            difference_save_widget.value,
            metrics_save_widget.value,
            energy_save_widget.value,
        ])
        save_dir = _save_dir_if_needed(needs_dir)
        save_json_path = (
            save_dir / "benchmark_metrics.json"
            if legacy_enabled and metrics_save_widget.value
            else None
]}  адп to expand need continuing. But response truncated? Wait newCode truncated to `else None
]} ???. We need to finish rest of cell. The new code truncated due to char limit? the snippet ended mid code near `else None

E_MeV =  3161420.018614536
Gamma =  3368.997759324243
E_rest =  938.385948501337
E_MeV =  -3161367.0952732796
Gamma =  16.40887837053219
E_rest =  193020.75925874696


E_MeV =  3161420.018614536
Gamma =  3368.997759324243
E_rest =  938.385948501337
E_MeV =  -3161367.0952732796
Gamma =  16.40887837053219
E_rest =  193020.75925874696
